In [ ]:
import data
import torch
from utils.distmat import *
from utils.evaluation import *
from hitl import *
import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [ ]:
key = data.get_output_keys()[2]
key

In [ ]:
output = data.load_output(key)
qf = torch.Tensor(output["qf"])
gf = torch.Tensor(output["gf"])
q_pids = np.array(output["q_pids"])
g_pids = np.array(output["g_pids"])
q_camids = np.array(output["q_camids"])
g_camids = np.array(output["g_camids"])
distmat = compute_distmat(qf, gf)

### Baseline Results

In [ ]:
result = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)
result

### Re-ranked Results

In [ ]:
all_distmat = compute_inner_distmat(torch.cat((qf, gf)))
re_distmat = rerank_distmat(all_distmat, qf.shape[0])
re_result = evaluate(re_distmat, q_pids, g_pids, q_camids, g_camids)
re_result

## One-Shot Evaluation Using Modules

### Rocchio

In [ ]:
ntot = torch.as_tensor
rocchio.run(qf, gf, ntot(q_pids), ntot(g_pids), ntot(q_camids), ntot(g_camids), t=3)

### Neighborhood Expansion (Min)

In [ ]:
ntot = torch.from_numpy
ne.run(qf, gf, ntot(q_pids), ntot(g_pids), ntot(q_camids), ntot(g_camids), t=3, method="min")

### Neighborhood Expansion (Mean)

In [ ]:
ntot = torch.from_numpy
ne.run(qf, gf, ntot(q_pids), ntot(g_pids), ntot(q_camids), ntot(g_camids), t=3, method="mean")

## Development

## Feedback Models

In [ ]:
q_pids = torch.tensor(output["q_pids"])
g_pids = torch.tensor(output["g_pids"])
q = len(q_pids)
g = len(g_pids)
m = qf.shape[1]
q_camids = np.array(output["q_camids"])
g_camids = np.array(output["g_camids"])

### Naive Feedback

In [ ]:
if input("reset? ") == "y":
    positive_indices = torch.zeros((q, g), dtype=bool)
    negative_indices = torch.zeros((q, g), dtype=bool)

In [ ]:
for i in tqdm(range(5)):
    qf_adjusted = qf  # no adjust, naive re-rank

    distmat = compute_distmat(qf_adjusted, gf)
    distmat[positive_indices] = float("inf")
    distmat[negative_indices] = float("inf")

    # Select feedback (top-1 from remaining gallery instances)
    distances, indices = distmat.min(dim=1)
    assert(tuple(distances.shape) == (q,))
    assert(tuple(indices.shape) == (q,))

    pmap = g_pids[indices] == q_pids
    positive_q = torch.arange(0, q)[pmap]
    negative_q = torch.arange(0, q)[pmap == False]
    positive_g = indices[pmap]
    negative_g = indices[pmap== False]

    existing = positive_indices[positive_q, positive_g]
    assert(not existing.any())
    positive_indices[positive_q, positive_g] = True
    existing = negative_indices[negative_q, negative_g]
    assert(not existing.any())
    negative_indices[negative_q, negative_g] = True

In [ ]:
distmat = compute_distmat(qf_adjusted, gf)
distmat[positive_indices] = 0
distmat[negative_indices] = float("inf")
naive_new_result = evaluate(distmat.numpy(), q_pids, g_pids, q_camids, g_camids)
naive_new_result

### Rocchio

In [ ]:
if input("reset? ") == "y":
    positive_indices = torch.zeros((q, g), dtype=bool)
    negative_indices = torch.zeros((q, g), dtype=bool)

In [ ]:
alpha = 1
beta = 0.65
gamma = 0.35
qf_adjusted = qf

for i in tqdm(range(5)):
    distmat = compute_distmat(qf_adjusted, gf)
    distmat[positive_indices] = float("inf")
    distmat[negative_indices] = float("inf")

    # Select feedback (top-1 from remaining gallery instances)
    distances, indices = distmat.min(dim=1)
    assert(tuple(distances.shape) == (q,))
    assert(tuple(indices.shape) == (q,))

    # Apply feedback
    pmap = g_pids[indices] == q_pids
    positive_q = torch.arange(0, q)[pmap]
    negative_q = torch.arange(0, q)[pmap == False]
    positive_g = indices[pmap]
    negative_g = indices[pmap== False]

    existing = positive_indices[positive_q, positive_g]
    assert(not existing.any())
    positive_indices[positive_q, positive_g] = True
    existing = negative_indices[negative_q, negative_g]
    assert(not existing.any())
    negative_indices[negative_q, negative_g] = True
    
    # Compute new query
    mean_positive_gf = positive_indices.float().mm(gf) / positive_indices.float().sum(dim=1, keepdim=True)
    mean_negative_gf = negative_indices.float().mm(gf) / negative_indices.float().sum(dim=1, keepdim=True)
    mean_positive_gf[mean_positive_gf.isnan()] = 0
    mean_negative_gf[mean_negative_gf.isnan()] = 0
    qf_adjusted = qf * alpha + mean_positive_gf * beta - mean_negative_gf * gamma

In [ ]:
distmat = compute_distmat(qf_adjusted, gf)
distmat[positive_indices] = 0
distmat[negative_indices] = float("inf")
new_result = evaluate(distmat.numpy(), q_pids, g_pids, q_camids, g_camids)
new_result

### Function Tests (Rocchio)

In [ ]:
def adjust_qf(qf, gf, positive_indices, negative_indices, alpha=1, beta=0.65, gamma=0.35):
    assert(qf.shape[1] == gf.shape[1])
    mean_positive_gf = positive_indices.float().mm(gf) / positive_indices.float().sum(dim=1, keepdim=True)
    mean_negative_gf = negative_indices.float().mm(gf) / negative_indices.float().sum(dim=1, keepdim=True)
    mean_positive_gf[mean_positive_gf.isnan()] = 0
    mean_negative_gf[mean_negative_gf.isnan()] = 0
    qf_adjusted = qf * alpha + mean_positive_gf * beta - mean_negative_gf * gamma
    return qf_adjusted

In [ ]:
def update_feedback_indices(distmat, q_pids, g_pids, positive_indices, negative_indices, inplace=True):
    """
    Note that distmat is corrupted if inplace=True.
    
    distmat: q x g Tensor (adjusted query to gallery)
    q_pids: q
    g_pids: g
    positive_indices: q x g
    negative_indices: q x g
    
    :Returns:
        positive_indices, negative_indices
    """
    q, g = tuple(distmat.shape)
    
    if not inplace:
        distmat = distmat.clone().detach()
        positive_indices = positive_indices.copy()
        negative_indices = negative_indices.copy()
        
    distmat[positive_indices] = float("inf")
    distmat[negative_indices] = float("inf")
        
    indices = distmat.argmin(dim=1)
    pmap = g_pids[indices] == q_pids
    positive_q = torch.arange(0, q)[pmap]
    negative_q = torch.arange(0, q)[pmap == False]
    positive_g = indices[pmap]
    negative_g = indices[pmap== False]

    existing = positive_indices[positive_q, positive_g]
    assert(not existing.any())
    positive_indices[positive_q, positive_g] = True
    existing = negative_indices[negative_q, negative_g]
    assert(not existing.any())
    negative_indices[negative_q, negative_g] = True
    
    return positive_indices, negative_indices

In [ ]:
def init_feedback_indices(q, g):
    return torch.zeros((q, g), dtype=bool)

In [ ]:
def update_distmat(qf, gf, q_pids, g_pids, positive_indices=None, negative_indices=None,
                   inplace=True, previous_distmat=None, alpha=1, beta=0.65, gamma=0.35):
    """
    previous_distmat: adjusted distmat (!= compute_distmat(qf, gf))
    """
    q, g = qf.shape[0], gf.shape[0]
    assert(qf.shape[1] == gf.shape[1])
    
    if positive_indices is None:
        positive_indices = init_feedback_indices(q, g)
    if negative_indices is None:
        negative_indices = init_feedback_indices(q, g)

    distmat = previous_distmat 
    if distmat is None:
        qf_adjusted = adjust_qf(qf, gf, positive_indices, negative_indices)
        distmat = compute_distmat(qf_adjusted, gf)

    positive_indices, negative_indices = update_feedback_indices(
        distmat, q_pids, g_pids, positive_indices, negative_indices, inplace=inplace)
    
    qf_adjusted = adjust_qf(qf, gf, positive_indices, negative_indices, alpha=alpha, beta=beta, gamma=gamma)
    distmat = compute_distmat(qf_adjusted, gf)
    
    return distmat, positive_indices, negative_indices

In [ ]:
positive_indices = None
negative_indices = None
distmat = None
for i in tqdm(range(5)):
    distmat, positive_indices, negative_indices = update_distmat(
        qf, gf, q_pids, g_pids, positive_indices, negative_indices, previous_distmat=distmat)
    

In [ ]:
distmat[positive_indices] = 0
distmat[negative_indices] = float("inf")
new_result = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)
new_result

### Module Test (Naive)

In [ ]:
positive_indices = None
negative_indices = None
distmat = None
for i in tqdm(range(3)):
    distmat, positive_indices, negative_indices = feedback.naive_round(
        qf, gf, q_pids, g_pids, positive_indices, negative_indices, previous_distmat=distmat)
naive_result = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)
naive_result

### Module Test (Rocchio)

In [ ]:
positive_indices = None
negative_indices = None
distmat = None
for i in tqdm(range(3)):
    distmat, positive_indices, negative_indices = rocchio.rocchio_round(
        qf, gf, q_pids, g_pids, positive_indices, negative_indices, previous_distmat=distmat)
rocchio_result = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)
rocchio_result

### Single Feedback Rocchio (Old)
Initial implementation test

In [ ]:
g_pids = torch.Tensor(output["g_pids"])
q_pids = torch.Tensor(output["q_pids"])
match = g_pids[min_indices] == q_pids

In [ ]:
selected_gf = gf[min_indices]
selected_gf.shape

In [ ]:
weights = match.float() * (beta + gamma) - gamma

In [ ]:
weighted_feedback = selected_gf * weights.reshape(-1, 1)

In [ ]:
weighted_feedback

In [ ]:
inverse_weights = 1 - weights

In [ ]:
new_qf = qf * inverse_weights.reshape(-1, 1) + weighted_feedback

In [ ]:
new_distmat = compute_distmat(new_qf, gf)

In [ ]:
new_result = evaluate(new_distmat.numpy(), q_pids, g_pids, np.array(output["q_camids"]), np.array(output["g_camids"]), test_ratio=0.1)
new_result